In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from pathlib import Path
import collections 
import numpy as np
import json
import dask.dataframe as dd
import time
import zipfile as zfile
import threading
import datetime as dt

In [3]:
# 取得壓縮檔內的檔名

df_fn = []
zf = zfile.ZipFile('../POSTDB.zip')
for fname in zfile.ZipFile.namelist(zf):
    df_fn.append(fname.encode('cp437').decode('big5'))
df_fn

['CC.csv',
 'CC.SQL',
 'GPS.SQL',
 'GPSS1.CSV',
 'TT.SQL',
 'TTS1.csv',
 '中華郵政大數據競賽資料欄位規格.xlsx',
 '中華郵政大數據競賽資料欄位規格_新.xlsx',
 'ORACLE相關/',
 'ORACLE相關/ACC_Oracle.SQL',
 'ORACLE相關/CC_Oracle.SQL',
 'ORACLE相關/createDB.txt',
 'ORACLE相關/GPS_Oracle.SQL',
 'ORACLE相關/sqlldr.exe',
 'ORACLE相關/TTS1.CTL',
 'ORACLE相關/TT_Oracle.SQL',
 '培訓暨說明會 - 簡報檔/',
 '培訓暨說明會 - 簡報檔/1.競賽辦法說明.pdf',
 '培訓暨說明會 - 簡報檔/2.競賽資料說明-1.pdf',
 '培訓暨說明會 - 簡報檔/2.競賽資料說明-2(中華郵政大數據競賽資料欄位規格).xlsx',
 '培訓暨說明會 - 簡報檔/3.郵務知識培訓-1.pdf',
 '培訓暨說明會 - 簡報檔/3.郵務知識培訓-2.pdf',
 '培訓暨說明會 - 簡報檔/4.微軟Power BI分析軟體介紹.pdf',
 '培訓暨說明會 - 簡報檔/5.競賽命題及方向之工研院楊奇達經理建議.pdf',
 '培訓暨說明會 - 簡報檔/5.競賽命題及方向之高端訓博士建議.pdf',
 'ACC.SQL',
 'ACCS1.csv']

In [21]:
# 由壓縮檔取得所需要資料檔

zf = zfile.ZipFile('../POSTDB.zip')
date_spec = {'Mail_datetime': [2, 3]}
# dateparse = lambda x: pd.datetime.strptime("Mail_date" + " " + "Mail_time", '%Y-%m-%d %H:%M:%S')
# mydateparser = lambda x: pd.datetime.strptime(date_spec, "%Y-%m-%d %H:%M:%S")
dateparse = lambda x: pd.to_datetime('Mail_datetime', format='%Y-%m-%d %H:%M:%S',errors='ignore')
df = pd.read_csv(zf.open('TTS1.csv'), # 檔案位置
                 header=None, # 首行是否有欄位名稱
                 encoding="big5", # 編碼
                 dtype = "str", # 資料類型
                 nrows = 10000000, #讀取行數
                 names = ["Status_code", "Mail_num", "Mail_date", "Mail_time", "OP_office", "other"] # 欄位名稱
                )

In [22]:
df.groupby(['Mail_num']).size().reset_index(name='counts').sort_values(['counts'], ascending=False)

,Mail_num,counts
0,00000000000000,23
3304399,80077700104470,22
2753568,38163710006817222005,15
1388756,06668100104770,12
928879,03258850000070,12
928897,03258950000070,12
928915,03259050000070,12
3272586,76800700103870,12
4051888,95634500103370,12
3070546,59033110009118,11


In [23]:
df.head(5)

,Status_code,Mail_num,Mail_date,Mail_time,OP_office,other
0,Y4,00000000000000,2018-01-01,09:49:04,330031,003
1,Y4,00000000000000,2018-01-01,09:58:08,330031,002
2,I4,00000000000000,2018-01-01,14:11:51,330031,
3,Y4,00000000000000,2018-01-01,14:23:32,330031,006
4,I4,00000000000000,2018-01-01,14:52:09,330031,


In [24]:
print(type(df.columns[0]))

<class 'str'>


In [25]:
# TT1 郵件狀態代碼 X(2) | Status_code
# TT2 掛號號碼 X(20) | Mail_num
# TT3 處理日期 X(10) | Mail_date
# TT4 處理時間 X(8) | Mail_time
# TT5 處理局號 X(6) | OP_office // operation office
# TT6 其它 X(42) | other

df.columns = ["Status_code", "Mail_num", "Mail_date", "Mail_time", "OP_office", "other"]

# 定義：狀態碼為 **點**

In [26]:
nodes = list(df.Status_code.unique())

In [27]:
head_nodes = sorted(list(set([ node[0] for node in nodes])))

In [28]:
dic_nodes  = dict( zip(head_nodes, range(len(head_nodes))) )

In [29]:
label_node = lambda x: dic_nodes[x[0]]

In [30]:
# 只有狀態為節點
data_nodes = []
for node in nodes:
    data = { "id": node, "group": label_node(node)}
    data_nodes.append(data)

# 定義：**線**，狀態的改變
- 要依據狀態、郵件號碼及時間去決定**線** 的連接
- 時間的轉換，請參考 [pandas.to_datetime()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

In [31]:
df['Mail_datetime'] = df.Mail_date+" "+df.Mail_time

In [32]:
df.Mail_datetime = pd.to_datetime(df.Mail_datetime)

```
all_mail["58668700100170"] = [ ("Y4", "2018-01-01 09:49:04"),
                           ("I4", "2018-01-01 14:11:51"), ... ] 

```

In [33]:
df['Code_OP'] = df.Status_code + "-" + df.OP_office.map(str) 

In [34]:
data_nodes = []
for node in df['Code_OP'].unique():
    data = { "id": node, "group": label_node(node)}
    data_nodes.append(data)

In [35]:
start_time = time.time()
all_mail = {}
all_mail_key = set()
for idx, row in df.iterrows():
    mail_code = row.Mail_num.strip()
    if not mail_code in all_mail_key:
        all_mail[mail_code] = []
        all_mail_key.add(mail_code)
        
    all_mail[mail_code].append( (row.Code_OP, row.Mail_datetime) )
    

time_diff = time.time() - start_time
print("處理 ",len(df), "筆資料，組成 ", len(all_mail.keys()), " 筆資料，共執行：", time_diff, "秒")

處理  10000000 筆資料，組成  4419944  筆資料，共執行： 765.3743736743927 秒


In [83]:
len(all_mail.keys())

4419944

In [84]:
all_mail['00120000014101061010']

[('Y4-500584', Timestamp('2018-01-02 07:50:39')),
 ('H4-500584', Timestamp('2018-01-02 14:06:59')),
 ('Z2-500584', Timestamp('2018-01-02 16:34:28'))]

In [131]:
def convert_2_edge(mail_status):
    edges = []
    for idx in range(len(mail_status)-1):
        edges.append( ( mail_status[idx][0], mail_status[idx+1][0]) )
        
    return edges
# "Y4" -> "H4", 
# "H4" -> "Z2"

mail_status = all_mail['96410700000070']
convert_2_edge(mail_status)

[('Y4-540028', 'Y4-540028'),
 ('Y4-540028', 'I4-540028'),
 ('I4-540028', 'I4-540028')]

In [151]:
all_edges = [] 
for mail_code in all_mail:
    status_num = len(all_mail[mail_code])
    
    if (status_num) > 1:
        mail_status = all_mail[mail_code]
        all_edges.extend(convert_2_edge(mail_status))

value_list = {}
for edge, value in collections.Counter(all_edges).most_common(1000):
    value_list[edge] = value

list_max = np.max(list(value_list.values()))
list_min = np.min(list(value_list.values()))
list_diff = float(list_max - list_min)

k = 20

normal_val = lambda x: int(k/3+k*2*(x-list_min)/list_diff)

data_edges = []
for edge, value in collections.Counter(all_edges).most_common(1000):
    edge = {"source": edge[0], "target":edge[1], "value":normal_val(value)}
    data_edges.append(edge)

all_data = { "nodes":data_nodes, "links":data_edges}

open("data_4_d3.json", 'w').write(json.dumps(all_data, indent=2))

1133859